# Campaign Winner Processing

In [109]:
import csv
import os
import json
import tweepy
import requests
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np

In [320]:
#import file downloaded from gleam
#redundancy removed already
df = pd.read_csv(r'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\distinctcertikbnb2.csv')
df


,Name_,FirstName_,Last_name,Email_,Status_,Entries_,City_,Region_,Country_,Created_At,Twitter_
0,startiktik485 patel,startiktik485,patel,startiktok485@gmail.com,Valid,1,Indore,MP,India,1/14/2021 0:00,https://twitter.com/startiktik485\\r
1,Renee Saunders,Renee,Saunders,ninas.sy27@gmail.com,Valid,1,Ho Chi Minh City,SG,Vietnam,1/14/2021 0:00,https://twitter.com/jannebastoss\\r
2,sp582757 kas,sp582757,kas,sp582757@gmail.com,Valid,1,Indore,MP,India,1/14/2021 0:00,https://twitter.com/i_sam1452\\r
3,patelvisal330 patel,patelvisal330,patel,patelvisal330@gmail.com,Valid,1,Bhopal,MP,India,1/14/2021 0:00,https://twitter.com/patelvisal330\\r
4,jsohan6166 patel,jsohan6166,patel,jsohan616@gmail.com,Valid,1,Indore,MP,India,1/14/2021 0:00,https://twitter.com/jsohan6166\\r
...,...,...,...,...,...,...,...,...,...,...,...
1086,M K,M,K,cryptoskeep@gmail.com,Valid,1,Wattenberg,7,Austria,1/5/2021 0:00,https://twitter.com/JooopJopp\\r
1087,Crypto Junior,Crypto,Junior,curateku@gmail.com,Valid,1,Jakarta,JK,Indonesia,1/5/2021 0:00,https://twitter.com/Cryptojr7\\r
1088,Jose Lucas Bazan,Jose,Lucas Bazan,joselubazan544@gmail.com,Valid,1,Vinhedo,SP,Brazil,1/5/2021 0:00,https://twitter.com/joselubazan\\r
1089,Geh Cha Kai,Geh,Cha Kai,castergeh@gmail.com,Valid,1,Kuala Lumpur,14,Malaysia,1/5/2021 0:00,https://twitter.com/Geh_Cha_Kai\\r


In [321]:
df['Twitter_'].head(3)

0    https://twitter.com/startiktik485\\r
1     https://twitter.com/jannebastoss\\r
2        https://twitter.com/i_sam1452\\r
Name: Twitter_, dtype: object

In [322]:
#clean the string data
df['Twitter_'] = df['Twitter_'].replace({'\\r':''}, regex=True)
df['Twitter_'] = df['Twitter_'].replace({'https://twitter.com/':''}, regex=True)
df['Twitter_'].head(3)



0    startiktik485\
1     jannebastoss\
2        i_sam1452\
Name: Twitter_, dtype: object

In [323]:
#wiredly cannot remove the \ in the previous code
df['Twitter_'] = df.Twitter_.str[0:-1]
df['Twitter_'].head(3)

0    startiktik485
1     jannebastoss
2        i_sam1452
Name: Twitter_, dtype: object

In [324]:
#covert the dataframe column to a list 
twitterlist = df['Twitter_'].values.tolist()
twitterlist


['startiktik485',
 'jannebastoss',
 'i_sam1452',
 'patelvisal330',
 'jsohan6166',
 'roOick',
 'kadirpatel16',
 'samarsiji5',
 'MU_S_T_Y',
 'ippatel297',
 'kadirbh79038938',
 'duvidha3',
 'hijikaji',
 'patelisrael328',
 'samanahe1',
 '82Evans',
 'Candied110',
 'na_aasu',
 'aasiyanajee1',
 'olz00ah',
 'patelisrael290',
 'samar36610942',
 'dmkkk',
 'BhajiSamar',
 'samar_ti',
 'AasikJi',
 'all_over2008',
 'jisamar778',
 'LuzaTutifruti',
 'AsheeMore',
 'chocotofcaramel',
 '46ihsans',
 'maxiyprin',
 'polita2206',
 'ceylanylmaz8',
 '13abAndino',
 'AylaDA4',
 'CLAUGAVIOTA',
 'missind70',
 'samajbd',
 'szgnesn',
 'Picarinesco',
 'FalcoGM',
 'litsavl7',
 'fanesa_caca',
 'mariento',
 'PEDROALMEIDARO3',
 'dougEz',
 'patelisrael24',
 '272698800',
 'QarizmatiqAsik',
 'n90_oliveira',
 'M0nti99',
 'shygirl1900',
 'bobboy10',
 'lemonskipper',
 'Yovisszoe',
 'ANOMAX23',
 'eMMa_R_r',
 'BAHARglr',
 '3214579325',
 'Apocaliptico09',
 'bradley27887',
 'screwy89',
 'hunter_28',
 'OSOJHOU',
 'Abdo_Eissaa',
 'a

# Using Twitter Api

In [108]:
#auth
consumer_key = 'wwdiE5ykBjzTchyiRsaBoTlsM'
consumer_secret = 'Yvhc1WwAMecWRXx7Zavyzr9L1YhpkyywhbHCs5LEoniO6FrB6H'
access_token = '805275096713674752-66JCPlGs5y01sjlTcazy1v0NTR2K3pZ'
access_token_secret = 'yhdD0Xakqw4qHKatJNo1hy4ud3vHCImxAA74Gp8A2eaW1'

# OAuthHandler using the keys and tokens
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# authentication api
api = tweepy.API(auth, wait_on_rate_limit=True)

In [114]:
#test
user_info = api.get_user('certikorg')._json
follower = user_info['followers_count']
#total_tweets = user_info['statuses_count']
print (follower)

11833


In [327]:
df = df.rename(columns={'Twitter_': 'Participants'})
df.head()


,Name_,FirstName_,Last_name,Email_,Status_,Entries_,City_,Region_,Country_,Created_At,Participants
0,startiktik485 patel,startiktik485,patel,startiktok485@gmail.com,Valid,1,Indore,MP,India,1/14/2021 0:00,startiktik485
1,Renee Saunders,Renee,Saunders,ninas.sy27@gmail.com,Valid,1,Ho Chi Minh City,SG,Vietnam,1/14/2021 0:00,jannebastoss
2,sp582757 kas,sp582757,kas,sp582757@gmail.com,Valid,1,Indore,MP,India,1/14/2021 0:00,i_sam1452
3,patelvisal330 patel,patelvisal330,patel,patelvisal330@gmail.com,Valid,1,Bhopal,MP,India,1/14/2021 0:00,patelvisal330
4,jsohan6166 patel,jsohan6166,patel,jsohan616@gmail.com,Valid,1,Indore,MP,India,1/14/2021 0:00,jsohan6166


In [135]:
#get a list of participants' followers, skip the errors like "cannot finds user" or "account suspended"
followers = []
id_list = []
#print(user_info)
for id in twitterlist:
    try:
        user_info = api.get_user(id)._json
        follower = user_info['followers_count']
        followers.append(follower)
        id_list.append(id)
    except Exception as e:
        pass
    continue
print (id_list)
print(followers)

['jannebastos', 'roOic', 'kadirpatel1', 'samarsiji', 'samanahe', '82Evan', 'Candied11', 'na_aas', 'dmkk', 'samar_t', 'AasikJ', '46ihsan', 'polita220', 'ceylanylmaz', 'AylaDA', 'samajb', 'szgnes', 'marient', 'dougE', 'shygirl190', 'bobboy1', 'ANOMAX2', 'eMMa_R_', 'BAHARgl', '321457932', 'screwy8', 'hunter_2', 'OSOJHO', 'aydn_k', 'pacanchik013_', 'CavitKARA', 'darioezequiel1', 'Manuelgonzz', 'fabrizzi0', 'MetalCu', 'pikirin', 'ivanamit', 'libho', 'manuelita_15', 'ahmadkus', 'a_ya_', 'AlicanBe', 'viny123', 'enginayaz', 'BeyzaBs', 'ferystar', 'Tpatala', 'levent1979', 'kraken2', 'aassu1', 'samarsir', 'CryptoAngel1', 'whoopyhoop', 'Zondiak', 'Rahmatdony', 'flora046', 'josesantos5', 'RaZk7', 'quaresmam', 'crypto_dick', 'selcukcell', 'amandarosita1', 'emilylama', 'mattu8', 'pantera289', 'Juasanv', 'muharrem9', 'MarleSk', 'Franph', 'dawidq', 'israelpatel1', 'ahmedsayed6', 'vntricky', 'Behruzjaan', 'CryptoLand1', 'Cryptoshoes', 'Rajkumaranna', 'GustavoAscenzi', 'sanamre0', 'rabo110', 'azgarkhan1

In [315]:
#Check length
print(len(id_list))
print(len(followers))

342
342


In [280]:
#Create Dataframe for participants with active twitter account
dict = {'Participants':id_list, 'Followers':followers }
df_valid = pd.DataFrame(dict)
df_valid

,Participants,Followers
0,jannebastos,1
1,roOic,0
2,kadirpatel1,0
3,samarsiji,0
4,samanahe,36
...,...,...
337,kik_gar,0
338,Rjramesh,1
339,AlexKosa,6
340,Shariful100,0


In [291]:
#rank
df_valid = df_valid.sort_values(by=['Followers'], ascending = False)
df_valid.head(50)

,Participants,Followers
281,tavria,3547
229,bacodsan_tuy,2672
18,dougE,1471
43,enginayaz,954
203,ferinomore,731
35,pikirin,723
103,qrtrmstr,481
231,VertFade,468
302,Renat8,394
317,desaihira,326


In [ ]:
#save the data 
#end